# 제품 이상여부 판별 프로젝트


### 데이터 읽어오기


In [1]:
import pandas as pd

RANDOM_STATE = 110

train_data = pd.read_csv("../../../data/train_data_0816.csv")
test_data = pd.read_csv("../../../data/test_data_0816.csv")

In [2]:
# 공통 변수 리스트
com_variables_train = [
    'target', 'Model.Suffix', 'Workorder', 'WorkMode Collect Result'
    , 'Dispenser_1', 'Dispenser_2', 'Receip_No_Collect_Result'
    , 'Production_Qty_Collect_Result', 'Judge_Value_OK'
    , 'Workorder_0.9', 'Workorder_0.6'
]

com_variables_test = [
    'target', 'Set ID', 'Model.Suffix', 'Workorder'
    , 'WorkMode Collect Result', 'Dispenser_1'
    , 'Dispenser_2', 'Receip_No_Collect_Result'
    , 'Production_Qty_Collect_Result', 'Judge_Value_OK'
    , 'Workorder_0.9', 'Workorder_0.6'
]

In [3]:
# 공정 이름 필터링 후 공통 변수와 결합
def create_dataset(train_data, test_data, process_name, com_variables_train, com_variables_test):
    # 열 이름 필터링
    Process_Desc_col = train_data.filter(like=process_name).columns
    
    # train 데이터셋 생성
    final_columns_train = list(Process_Desc_col) + com_variables_train
    train_dataset = train_data[final_columns_train]
    
    # test 데이터셋 생성
    final_columns_test = list(Process_Desc_col) + com_variables_test
    test_dataset = test_data[final_columns_test]
    
    return train_dataset, test_dataset

# 공통 변수 정의
## com_variables_train = [...]  -> 이전 코드에서 정의한 변수 사용
## com_variables_test = [...]   -> 이전 코드에서 정의한 변수 사용

# 데이터셋 생성
train_data_dam, test_data_dam = create_dataset(train_data, test_data, '_Dam', com_variables_train, com_variables_test)
train_data_fill1, test_data_fill1 = create_dataset(train_data, test_data, '_Fill1', com_variables_train, com_variables_test)
train_data_fill2, test_data_fill2 = create_dataset(train_data, test_data, '_Fill2', com_variables_train, com_variables_test)
train_data_autoclave, test_data_autoclave = create_dataset(train_data, test_data, '_AutoClave', com_variables_train, com_variables_test)

---

## Optuna

스레스홀드 0.3으로 맞춘상태에서 튜닝 진행한 것

In [ ]:
# import optuna
# from xgboost import XGBClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import f1_score

# # 'Normal'과 'AbNormal'을 숫자로 변환
# train_data_fill2['target'] = train_data_fill2['target'].map({'Normal': 0, 'AbNormal': 1})

# # 스레드홀드 설정
# THRESHOLD = 0.3

# def objectiveXGB(trial, x_tr, y_tr, x_val, y_val):
#     param = {
#         'n_estimators': trial.suggest_int('n_estimators', 1000, 2500)
#         , 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2)
#         , 'max_depth': trial.suggest_int('max_depth', 3, 10)

#         , 'alpha': trial.suggest_float('alpha', 0.0001, 0.01, log=True)
#         , 'gamma': trial.suggest_float('gamma', 0.0001, 0.01, log=True)

#         , 'reg_alpha' : trial.suggest_float('reg_alpha', 0.01, 1)
#         , 'reg_lambda' : trial.suggest_float('reg_lambda', 0.01, 1)
        
#         , 'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.01, 1)
#         , 'subsample' : trial.suggest_float('subsample', 0.01, 1)
#         , 'objective': 'binary:logistic'  # 이진 분류
#         , 'tree_method' : "exact"        
#         , 'random_state': RANDOM_STATE
#     }
       
#     model = XGBClassifier(**param)
#     model.fit(x_tr, y_tr)
#     pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
#     pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
#     score = f1_score(y_val, pred, average="binary")
    
#     return score

# # 데이터셋 분할
# x_train, x_val, y_train, y_val = train_test_split(
#     train_data_fill2.drop("target", axis=1),
#     train_data_fill2["target"],
#     test_size=0.2,
#     shuffle=True,
#     random_state=RANDOM_STATE,
# )

# # 하이퍼 파라미터 튜닝
# study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
# study.optimize(lambda trial: objectiveXGB(trial, x_train, y_train, x_val, y_val), n_trials=150)

# print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))


Best trial: score 0.23633677991137375,   
params {'n_estimators': 2395, 'learning_rate': 0.07582121344743285, 'max_depth': 5,  
'alpha': 0.00018378623188564677, 'gamma': 0.009548370334837984,  
'reg_alpha': 0.8773284617743917, 'reg_lambda': 0.4458568017060983,  
'colsample_bytree': 0.7870458340804413, 'subsample': 0.9104043107037911}

### 두번째 진행

In [ ]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# 'Normal'과 'AbNormal'을 숫자로 변환
train_data_fill2['target'] = train_data_fill2['target'].map({'Normal': 0, 'AbNormal': 1})

# 스레드홀드 설정
THRESHOLD = 0.3

def objectiveXGB(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1200, 3200)
        , 'learning_rate': trial.suggest_float('learning_rate', 0.002, 0.2)
        , 'max_depth': trial.suggest_int('max_depth', 3, 10)

        , 'alpha': trial.suggest_float('alpha', 0.00001, 0.01, log=True)
        , 'gamma': trial.suggest_float('gamma', 0.00001, 0.01, log=True)

        , 'reg_alpha' : trial.suggest_float('reg_alpha', 0.01, 1)
        , 'reg_lambda' : trial.suggest_float('reg_lambda', 0.01, 1)
        
        , 'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.01, 1)
        , 'subsample' : trial.suggest_float('subsample', 0.03, 1)
        , 'objective': 'binary:logistic'  # 이진 분류
        , 'tree_method' : "exact"        
        , 'random_state': RANDOM_STATE
    }
       
    model = XGBClassifier(**param)
    model.fit(x_tr, y_tr)
    pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data_fill2.drop("target", axis=1),
    train_data_fill2["target"],
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(lambda trial: objectiveXGB(trial, x_train, y_train, x_val, y_val), n_trials=300)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))


Best trial: score 0.24999999999999994,  
params {'n_estimators': 1954, 'learning_rate': 0.030591237713800463, 'max_depth': 10, 'alpha': 0.0016931254036924568,  
'gamma': 0.0005962753560731528, 'reg_alpha': 0.7743077481335676, 'reg_lambda': 0.11095536861382667, 'colsample_bytree': 0.9309000726172385,  
'subsample': 0.9979120610759441}

.